# ML Project

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sklearn.metrics as sklm
from sklearn.dummy import DummyClassifier
import random
import metoder
import pandas as pd

Importing dataset:

In [21]:
#path:
#file_path = r"C:\Users\vikto\Documents\Skole\Datadrevne_systemer\ML_Project\glass_data-4_lev.csv"
file_path_training = "../Datasett/train_set.csv"
file_path_test = "../Datasett/test_set.csv"
file_path_total_set =   "../Datasett/mushrooms.csv"
#importing file and print how the structure of data were inported. 
df_training = pd.read_csv(file_path_training)
df_test = pd.read_csv(file_path_test)
df_total = pd.read_csv(file_path_total_set)

exstra_drop_list = ['capSurface', 'gillSize', 'stalkRoot', 'capShape']
drop_list = ['capSurface', 'capColor', 'odor', 'gillSize', 'gillColor', 'stalkColorAboveRing', 'stalkColorBelowRing', 'sporePrintColor', 'population', 'habitat']

drop_list = drop_list + exstra_drop_list

df_training = df_training.drop(columns=drop_list)
df_test = df_test.drop(columns=drop_list)
df_total = df_total.drop(columns=drop_list)


#drop ID number
#df = df.drop(columns=['Id'])

print(df_test.__len__())
df_total.info()

KeyError: "['stakColorBelowRing'] not found in axis"

### Small exploration of data:

In [ ]:
# Check the number of unique rows in the DataFrame
num_unique_rows = df_total.drop_duplicates().shape[0]

# Print the result
print("Number of unique rows:", num_unique_rows)

The data set clearly shows that T1, T2 is most frequently. 

In [ ]:
# Create a histogram of age with hue for income
#sns.histplot(data=df_total, x='capShape', hue='class', multiple='stack', palette='pastel')

# Add labels and title
#plt.xlabel('cap shape')
#plt.ylabel('Frequency')
#plt.title('cap shape Distribution by class')

# Show the plot
#plt.show()

In [ ]:
#for col in df.columns[:-1]:  # Exclude the 'Type' column
 #   sns.boxplot(x='Type', y=col, data=df)
  #  plt.show()

One-hot encoding to ensure the best result

In [ ]:
# Perform one-hot encoding for all categorical columns except the "class" column
df_train_encoded = pd.get_dummies(df_training, columns=df_training.drop(columns=['class']).columns.tolist())[0:2000]
df_test_encoded = pd.get_dummies(df_test, columns=df_test.drop(columns=['class']).columns.tolist())

# Replace "p" with 1 and "e" with 0 in the "class" column in the df_training and df_test DataFrames
df_train_encoded['class'] = df_train_encoded['class'].apply(lambda x: 1 if x == 'p' else 0)
df_test_encoded['class'] = df_test_encoded['class'].apply(lambda x: 1 if x == 'p' else 0)

# Splitting into features (X) and target variable (y)
X_train, y_train = df_train_encoded.drop(['class'], axis=1), df_train_encoded['class']
X_test, y_test = df_test_encoded.drop(['class'], axis=1), df_test_encoded['class']

y_train = y_train.astype(int)
y_test = y_test.astype(int)

print(X_train.shape)
print(X_test.shape)

### Make a neural network:

Split data to training and validation and Scale the data: 

In [ ]:
# Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and get the training history
history = model.fit(X_train, y_train, epochs=5, batch_size=10, validation_data=(X_test, y_test))

# Predict
y_pred = model.predict(X_test)
# Convert probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype("int32")

# Plot the validation loss over epochs
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Evaluate model:

In [ ]:
# Results storage as a dictionary
results = {}

#neural network:
# Create confusion matrix and normalize it over predicted (columns)
cm1 = sklm.confusion_matrix(y_test, y_pred_binary)

# Calculate accuracy
accuracy1= sklm.accuracy_score(y_test, y_pred_binary)

results['neural network'] = {'accuracy': accuracy1, 'confusion_matrix': cm1}
print(accuracy1, cm1)

In [ ]:
# Call each function and store the results
accuracy2, cm2 = metoder.randomforest(X_train, X_test, y_train, y_test)
results['random forrest'] = {'accuracy': accuracy2, 'confusion_matrix': cm2}
print(accuracy2, cm2)

In [ ]:
accuracy3, cm3 = metoder.gradientboost(X_train, X_test, y_train, y_test)
print(accuracy3, cm3)
results['gradientboost'] = {'accuracy': accuracy3, 'confusion_matrix': cm3}

In [ ]:
#accuracy, confusion_matrix = metoder.linearregression(X_train, X_test, y_train, y_test)
#results['linearregression'] = {'accuracy': accuracy, 'confusion_matrix': confusion_matrix}


In [ ]:
accuracy4, cm4= metoder.decisionregressor(X_train, X_test, y_train, y_test)
results['decision tree classifier'] = {'accuracy': accuracy4, 'confusion_matrix':cm4}
print(accuracy4, cm4)

In [ ]:
#accuracy, cm = metoder.mlpregression(X_train, X_test, y_train, y_test)
#results['mlpregression'] = {'accuracy': accuracy, 'confusion_matrix': cm}

In [ ]:
accuracy5, cm5 = metoder.supportvectorclassifier(X_train, X_test, y_train, y_test)
results['supportvectoregressor'] = {'accuracy': accuracy5, 'confusion_matrix': cm5}
print(accuracy5, cm5)

In [ ]:

accuracy6, cm6 = metoder.kneighborsclassifier(X_train, X_test, y_train, y_test)
results['kneighborsregressor'] = {'accuracy': accuracy6, 'confusion_matrix': cm6}
print(accuracy6, cm6)

In [ ]:
#make tree lists of dictionary: 
algorithm_names = []
accuracies = []
confusion_matrices_list = []

for algorithm, results_info in results.items():
    algorithm_names.append(algorithm)
    accuracies.append(results_info['accuracy'])
    confusion_matrices_list.append(results_info['confusion_matrix'])


In [ ]:
compare_models = pd.DataFrame({ "Algorithms": algorithm_names, "accuracy": accuracies})
compare_models.sort_values(by = "accuracy", ascending = False)

In [ ]:
%matplotlib inline
plt.figure(figsize=(8,8))
sns.barplot(x = "accuracy", y = "Algorithms", data = compare_models)
plt.show()

In [ ]:
# Utility Evaluation 

ulitity_matrix = np.array([[1.5, -2],
                           [-0.2 , 0.7]])

print()
ulitity_score =[] 

for i in range(len(algorithm_names)):
    ulitity_score.append(np.sum(np.array(confusion_matrices_list[i])*ulitity_matrix))
    print(algorithm_names[i], ":", confusion_matrices_list[i], "utility score:", ulitity_score[i])
    

compare_models = pd.DataFrame({ "Algorithms": algorithm_names, "utility": ulitity_score})
compare_models.sort_values(by = "utility", ascending = False)

%matplotlib inline
plt.figure(figsize=(8,8))
sns.barplot(x = "utility", y = "Algorithms", data = compare_models)
plt.show()

